In [ ]:
import pandas as pd
artigo_treino = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Word2Vec/treino.csv")
artigo_teste = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Word2Vec/teste.csv")

In [ ]:
!python -m SpaCy download pt_core_news_sm

In [ ]:
import SpaCy
nlp = SpaCy.load("pt_core_news_sm")

In [ ]:
textos_para_tratamento = (titulos.lower() for titulos in artigo_treino.title)

In [ ]:
def trata_textos(doc):
  tokens_validos = []


  for token in doc:
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      tokens_validos.append(token.text)


  if len(tokens_validos) > 2:
    return " ".join(tokens_validos)

In [ ]:
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
  batch_size = 1000,
  n_process = -1)]

In [ ]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      vector_size = 300,
                      min_count = 5,
                      alpha = 0.03,
                      min_alpha = 0.007)

In [ ]:
titulos_tratados = titulos_tratados.dropna().drop_duplicates()

In [ ]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [ ]:
w2v_modelo.build_vocab(lista_lista_tokens)

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec

# iniciando a chamada callback
class callback(CallbackAny2Vec):
  def __init__(self):
    self.epoch = 0

  def on_epoch_end(self, model):
    loss = model.get_latest_training_loss()
    if self.epoch == 0:
        print('Loss após a época {}: {}'.format(self.epoch, loss))
    else:
        print('Loss após a época {}: {}'.format(self.epoch, loss- self.loss_previous_step))
    self.epoch += 1
    self.loss_previous_step = loss

In [ ]:
w2v_modelo.train(lista_lista_tokens,
  total_examples = w2v_modelo.corpus_count,
  epochs = 30,
  compute_loss = True,
  callbacks=[callback()])

In [ ]:
w2v_modelo_sg = Word2Vec(sg = 1,
  window = 5,
  vector_size = 300,
  min_count = 5,
  alpha = 0.03,
  min_alpha = 0.007)

w2v_modelo_sg.build_vocab(lista_lista_tokens)

w2v_modelo_sg.train(lista_lista_tokens,
total_examples = w2v_modelo_sg.corpus_count,
epochs = 30,
compute_loss = True,
callbacks=[callback()])

In [ ]:
w2v_modelo.wv.save_word2vec_format("/content/drive/MyDrive/Colab Notebooks/Word2Vec/modelo_cbow.txt", binary=False)
w2v_modelo_sg.wv.save_word2vec_format("/content/drive/MyDrive/Colab Notebooks/Word2Vec/modelo_sg.txt", binary=False)

In [ ]:
import SpaCy
nlp = SpaCy.load("pt_core_news_sm", disable=["parser", "ner", "tagger", "textcat"])

def tokenizador(texto):
  tokens_validos = []
  doc = nlp(texto)

  for token in doc:
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      tokens_validos.append(token.text.lower())

  return tokens_validos

texto = "Adoro a 342342 #$@#@#$ cidade de caldas novas!"
tokenizador(texto)

In [ ]:
import numpy as np

def combinacao_de_vetores_por_soma(palavras, modelo):
  vetor_resultante = np.zeros(300)
  for pn in palavras:
    try:
      vetor_resultante =+ modelo.wv.get_vector(pn)
    except KeyError:
      pass

  return vetor_resultante

def matriz_vetores(textos, modelo):
  x = len(textos)
  y = 300
  matriz = np.zeros((x,y))

  for i in range(x):
    palavras = tokenizador(textos.iloc[i])
    matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

  return matriz

In [ ]:
matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title, w2v_modelo)
matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title, w2v_modelo)
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)

matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_sg)
print(matriz_vetores_treino_sg.shape)
print(matriz_vetores_teste_sg.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(modelo, x_treino, y_treino, x_teste, y_teste):
  RL = LogisticRegression(max_iter = 800)
  RL.fit(x_treino, y_treino)
  categorias = RL.predict(x_teste)
  resultados = classification_report(y_teste, categorias)
  print(resultados)
  return RL

In [ ]:
RL_cbow = classificador(w2v_modelo,
  matriz_vetores_treino_cbow,
  artigo_treino.category,
  matriz_vetores_teste_cbow,
  artigo_teste.category)

RL_cbow = classificador(w2v_modelo_sg,
  matriz_vetores_treino_sg,
  artigo_treino.category,
  matriz_vetores_teste_sg,
  artigo_teste.category)